# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [4]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [5]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI(api_key = api_key)

API key looks good so far


In [6]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [7]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [14]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [15]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [16]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [17]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [18]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/perplexity-ai/r1-1776',
 '/deepseek-ai/DeepSeek-R1',
 '/microsoft/OmniParser-v2.0',
 '/Wan-AI/Wan2.1-T2V-14B',
 '/microsoft/wham',
 '/models',
 '/spaces/nanotron/ultrascale-playbook',
 '/spaces/Wan-AI/Wan2.1',
 '/spaces/lllyasviel/LuminaBrush',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/microsoft/OmniParser-v2',
 '/spaces',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/facebook/natural_reasoning',
 '/datasets/SakanaAI/AI-CUDA-Engineer-Archive',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Wri

In [23]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://claude.ai/',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/claude-3-7-sonnet',
 '/claude/sonnet',
 '/news/visible-extended-thinking',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/solutions/coding',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosu

In [24]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [26]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [27]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
perplexity-ai/r1-1776
Updated
7 days ago
•
13.7k
•
1.78k
deepseek-ai/DeepSeek-R1
Updated
2 days ago
•
4.58M
•
10.3k
microsoft/OmniParser-v2.0
Updated
8 days ago
•
5.89k
•
968
Wa

In [40]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [31]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 5,000 characters
    return user_prompt

In [32]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company profile', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nperplexity-ai/r1-1776\nUpdated\n7 days ago\n•\n13.7k\n•\n1.78k\ndeepseek-ai/DeepSeek-R1\nUpdated\n2 days ago\n•\n4.58M\n•\n10.3k\nmicrosoft/OmniParser-v2.0\nUpdated\n8 days ago\n•\n5.89k\n•\n968\nWan-AI/Wan2.1-T2V-14B\nUpdated\nabout 19 hours ago\n•\n17.2k\n•\n311\nmicrosoft/wham\nUpdated\n5 days ago\n•\n218\nBrowse 1M+ models\nSpaces\nRunning\n1.63k\n1.63k\nThe Ultra-Scale Playbook\n🌌\nThe ul

In [33]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [41]:
create_brochure("Interswitch", "https://interswitchgroup.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://interswitchgroup.com/about-us'}, {'type': 'careers page', 'url': 'https://www.interswitchgroup.com/career/'}, {'type': 'leadership page', 'url': 'https://interswitchgroup.com/leadership'}, {'type': 'corporate governance page', 'url': 'https://interswitchgroup.com/corporate-governance'}, {'type': 'corporate responsibility page', 'url': 'https://interswitchgroup.com/corporate-responsibility'}, {'type': 'contact page', 'url': 'https://interswitchgroup.com/contact-us'}]}


```markdown
# Welcome to the World of Interswitch: Where Payments Go from "Ughhh" to "Woohoo!" 🎉

**Hey there!** Whether you’re here looking to make life easier with digital payments, join the team, or invest in the next big fintech powerhouse, you’ve arrived at the right place! Let’s take a humorous dive into what makes Interswitch the place to be. 

---

## Who Are We? 
We are **Interswitch**, Africa’s superhero of electronic payments and digital commerce solutions! Founded in 2002, we've been saving the day with innovative solutions that make everyday payments look like a walk in the park. 🌳

### 🚀 Our Mission:
To **inspire Africa to greatness** through innovation, value creation, and a sprinkle of fintech magic! 💫

### **Core Values!**
- **Customer Success:** Because we like happy customers (they're like unicorns, rare but magical!)
- **Empowered People:** We trust our people to make the best decisions... like letting you take the last slice of pizza.
- **Open-minded Collaboration:** Everyone’s ideas are welcome, even those that involve cat memes.
- **Innovation & Excellence:** Always pushing the boundaries like we’re trying to win a gold medal in the Olympics of fintech!

---

## What Do We Do? 
We make payments so easy you can do it with one hand... while holding a snack in the other! 🥨

### **Our Awesome Services:**
- **Quickteller:** Pay your bills or buy funky socks! Yep, you can do it all here!
- **Verve Cards:** Our card scheme is so popular, even your grandma has one (and she swipes it like a pro).
- **SMEs & Corporates:** We've got payment solutions for businesses bigger than your aunt’s holiday feasts!

---

## Meet Our Customers
From tiny startups to colossal corporations, we support thousands of businesses as they navigate the waters of the digital payment ocean. 🌊 Don’t worry; we’ve got life jackets! 

### Key Stats:
- **190,000**+ Active businesses transacting daily
- **50,000,000**+ Cards activated (seriously, it’s a lot – we’re like the card-carrying Uncle Sam of Africa!)
- **41,000**+ Quickteller Paypoint agents ready to aid your payment adventures.

---

## Join the Interswitch Family! 
At Interswitch, not only is work about making impactful transactions, but also about cooking up team spirit. **Did we mention there’s lunch on us?** 🍕 

### Why Work With Us?
- **Competitive compensation** that’ll make your wallet happy!
- **Learning and development** – because knowledge is your ultimate superpower!
- **Flexible work policies** – we get that sometimes your couch is your favorite office.

### A Sneak Peek into Our Culture
Imagine a place where working hard pairs with community potlucks, and where your ideas can fly higher than an eagle soaring through the skies! 🦅 

---

## Our Leadership
Meet the awesome leaders who guide this ship! 🚢 From engineers turned executives to finance wizards, we've got a diverse squad that drives us toward a cashless future. 

---

## Conclusion
Ready to dive into sugary solutions at Interswitch? Whether you’re looking to get started with your digital payment adventure, join a vibrant team, or invest in a brilliant future, we’re here making it happen—as smooth as a buttered pancake on a Sunday morning!

### Connect with Us:
**Ready to join the revolution or learn more? Let’s chat!** Or as we like to say – let’s switch it up!

---

**Interswitch: Empowering Africa, One Payment at a Time!** 🇳🇬💳
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [37]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [39]:
stream_brochure("Interswitch", "https://interswitchgroup.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://interswitchgroup.com/about-us'}, {'type': 'leadership page', 'url': 'https://interswitchgroup.com/leadership'}, {'type': 'corporate governance page', 'url': 'https://interswitchgroup.com/corporate-governance'}, {'type': 'careers page', 'url': 'https://www.interswitchgroup.com/career/'}, {'type': 'corporate responsibility page', 'url': 'https://interswitchgroup.com/corporate-responsibility'}, {'type': 'contact page', 'url': 'https://interswitchgroup.com/contact-us'}, {'type': 'news page', 'url': 'https://interswitchgroup.com/news'}, {'type': 'brands page', 'url': 'https://interswitchgroup.com/brands'}]}


# Interswitch Brochure

## About Us
**Interswitch** is a leading technology-driven company focused on digitizing payments and driving digital commerce across Africa. Established in 2002, we have played a crucial role in developing the payment ecosystem in Nigeria and beyond through our innovations and services that connect individuals, businesses, and communities seamlessly.

### Mission
To create technology solutions that connect and empower individuals, businesses, and communities across Africa.

### Vision
A prosperous Africa, driven by a seamless exchange of value and commerce.

### Core Values
- **Customer Success:** Our focus is on delivering value to our customers.
- **Empowered People:** We believe in trusting our employees to make informed decisions.
- **Ethical Conduct:** We maintain integrity in all our dealings.
- **Open-minded Collaboration:** We welcome diverse opinions and ideas.
- **Innovation Excellence:** We embrace creativity and practical applications.

## What We Do
Interswitch offers a comprehensive suite of payment solutions for various sectors, including:

### Everyday Payments for Individuals
- **Quickteller:** A platform allowing users to pay bills, buy airtime, and make purchases easily.
- **Quickteller Paypoint:** Offering customers numerous bill payment options.

### Payment Solutions for SMEs & Corporates
- **Business Transaction Processing:** Comprehensive services for payment processing, collections, and remittances.
- **Interswitch Payment Gateway:** Seamless integration of digital payments for businesses.
- **Interswitch Disbursements:** Facilitates financial transactions effortlessly.

### Industry-Specific Solutions
- **Interswitch Health, Lending Services, Energy, and Transport:** Tailored payment solutions to drive growth in essential sectors.

### Payment Cards & Tokens
Offering various payment cards, including the widely accepted **Verve Card.** 

## Customers
Interswitch serves a diverse clientele, from small businesses to large multinational corporations. Over **190,000** active businesses transact daily on our platform. Our solutions are designed to minimize payment barriers, ensuring businesses can serve their customers effectively.

### Global Presence
Interswitch operates extensively across Africa and has expanded its footprint into global markets, with operations in Nigeria, Kenya, Uganda, Germany, the Gambia, and the DRC. 

## Company Culture
At Interswitch, we foster an inclusive and dynamic work environment that values innovation, collaboration, and integrity. We believe in empowering our team members and encourage open communication across all levels to drive success and creativity.

### Careers
Join us in shaping the future of digital payments in Africa! We are constantly looking for talented and motivated individuals to join our team. Explore exciting career opportunities at Interswitch and be a part of our journey towards innovation and excellence.

[Explore Career Opportunities](link-to-careers-page)

## Corporate Responsibility
We are committed to corporate social responsibility through initiatives that promote financial inclusion and community development, aiming to inspire Africa towards greatness.

## Contact Us
Ready to explore opportunities or learn more about our solutions? Connect with us!  
[Contact Us](link-to-contact-page)

---

**Interswitch - Your Gateway to Africa's Payment Ecosystem.**  
Empowering everyday payments and digital commerce across the continent.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>